In [1]:
import requests
import lxml
import csv
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def _save_cars(data_car,name,csv_headers):
    out_file_name = '{x}_cars_alfa.csv'.format(x=name)
    
    with open(out_file_name, mode='w') as f:
        writer = csv.writer(f)
        writer.writerow(csv_headers)

        for data in data_car:
            #row = [str(getattr(article, prop)) for prop in csv_headers]
            row =data
            writer.writerow(row)

In [3]:
def returnLocationFake(data,i):
        dataClean=[cleanLocation(dataAux.get_text(),i) for dataAux in data]
        return list(dataClean)

def returnLocation(data):
        dataClean=[cleanLocation(dataAux.get_text()) for dataAux in data]
        return list(dataClean)
       
    
def returnData(data):
    dataClean=[dataAux.get_text() for dataAux in data]
    #print(dataClean)
    return list(dataClean)
def returnUrl(data):
    dataClean=[dataAux.get('href') for dataAux in data]
    #print(dataClean)
    return list(dataClean)

In [4]:
def cleanLocationFake(data,i):
    return data.split('-')[i]

def cleanLocation(data):
    return data.split('-')[0]

def cleanYear(data):
    Year=[]
    Km=[]
    dataClean=[dataAux.get_text() for dataAux in data]
    for i in range(len(dataClean)):
        if((i%2)==0):
            Year.append(dataClean[i])
    return list(Year)
def cleanKm(data):
    Km=[]
    dataClean=[dataAux.get_text() for dataAux in data]
    for i in range(len(dataClean)):
        if((i%2)!=0):
            #mirar esta linea
            Km.append(float(dataClean[i].split('Km')[0]))
    return list(Km)

In [5]:
def postDate(soup):
    dateAprox=soup.find('span',attrs={'ui-pdp-subtitle'}).get_text().split('· Publicado hace ')
    #print(dateAprox[1])
    return dateAprox[1]

In [11]:
def createDateTest(urls):
    Dates=[]
    for url in urls:
        response_=requests.get(url,verify=False)
        if (response_.status_code==200):
            #print('EXITO')
            soup_=BeautifulSoup(response_.text,'html.parser')
            Dates.append( postDate(soup_) )
        else:
            Dates.append('0')
    return Dates

In [12]:
def dataTransform(name,year_cantKm,amount,currenly,location,url):
        if True:
            name = returnData(name)
            url = returnUrl(url)
            #date
            date=createDateTest(url)
            year = cleanYear(year_cantKm)
            km = cleanKm(year_cantKm)
            amount = returnData(amount)
            currenly = returnData(currenly)
            ubication = returnLocation(location)
            print('--------------------')
               
            print('long name:',len(name))
                
            print('long year:',len(year))
            
            print('long km:',len(km))
            
            print('long amount:',len(amount))
                
            print('long currenly:',len(currenly))
            
            print('long ubication:',len(ubication))
            
            print('long date:',len(date))
                
            print('long currenly:',len(url))            
            
            print('--------------------')
            print(url)
            print('--------------------')

            return list(zip(name,year,km,amount,currenly,ubication,date,url))

        #return list(zip(name,year,km,amount,currenly,city,province,date,url))
        #return (name,year,km,amount,currenly,date,url)     
        #return list( zip(name,year,km,amount,currenly,date,url) )

In [14]:
sig_page='https://listado.mercadolibre.com.ar/peugeot-208#D[A:peugeot%20208,L:undefined]'
response = requests.get(sig_page)
data=[]
if (response.status_code==200):
        soup = BeautifulSoup(response.text,'html.parser')

        dom=lxml.etree.HTML(str(soup))

        DimL=int( dom.xpath('//div[@class="ui-search-pagination"]/ul/li[contains(@class,"andes-pagination__button andes-pagination__button--current")]/span')[0].text )

        DimF=int( soup.find('li',attrs={'andes-pagination__page-count'}).text.split(" ")[1] )
        
        n_f=soup.find('h1',attrs={'ui-search-breadcrumb__title'})
        name_file=n_f.get_text()
        #header=['name','year','km','amount','currenly','city','province','date','url']
        header=['name','year','km','amount','currenly','ubication','date','url']
        while(DimL<DimF):
            #while(DimL<6):

                print(DimL)
                print(sig_page)

                    #title
                name = soup.find_all('h2',attrs={'ui-search-item__title'}) 
                url = soup.find_all('a',attrs={'ui-search-result__content ui-search-link'})
                    #cant_km 
                    #year
                year_cantKm=soup.find_all('li',attrs={'ui-search-card-attributes__attribute'})
                    #price
                amount=soup.find_all('span',attrs={'price-tag-fraction'})
                currenly=soup.find_all('span',attrs={'price-tag-symbol'})
                #location
                location= soup.find_all('span',attrs={'ui-search-item__group__element ui-search-item__location shops-custom-secondary-font'})

                print(dataTransform(name,year_cantKm,amount,currenly,location,url)  )
                data.extend(dataTransform(name,year_cantKm,amount,currenly,location,url)  )
                ###############print(data)
                #data.append( dataTransfor(name,year_cantKm,amount,currenly,location,url) )
                
                time.sleep(2)
                dom=lxml.etree.HTML(str(soup))
                sig_page=dom.xpath('//div[@class="ui-search-pagination"]/ul/li[contains(@class,"andes-pagination__button andes-pagination__button--next")]/a')[0].get('href')

                response = requests.get(sig_page)
                soup = BeautifulSoup(response.text,'html.parser')

                DimL+=1

1
https://listado.mercadolibre.com.ar/peugeot-208#D[A:peugeot%20208,L:undefined]


/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/

/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/

--------------------
long name: 48
long year: 48
long km: 48
long amount: 50
long currenly: 50
long ubication: 48
long date: 48
long currenly: 48
--------------------
['https://auto.mercadolibre.com.ar/MLA-1159840697-peugeot-2022-208-16-like-pack-l20-_JM#position=1&search_layout=grid&type=item&tracking_id=e57925a6-dab3-4ba1-abfc-bbbfe279cecb', 'https://auto.mercadolibre.com.ar/MLA-1163146618-peugeot-2008-feline-no-208-cronos-308-pulse-nivus-ecosport-x-_JM#position=2&search_layout=grid&type=item&tracking_id=e57925a6-dab3-4ba1-abfc-bbbfe279cecb', 'https://auto.mercadolibre.com.ar/MLA-1163221677-peugeot-208-feline-16-tiptronic-0km-robayna-gf-_JM#position=3&search_layout=grid&type=item&tracking_id=e57925a6-dab3-4ba1-abfc-bbbfe279cecb', 'https://auto.mercadolibre.com.ar/MLA-1160761400-peugeot-207-xt-tiptronic-premium-208-307-gol-fox-ka-fit-up-x-_JM#position=4&search_layout=grid&type=item&tracking_id=e57925a6-dab3-4ba1-abfc-bbbfe279cecb', 'https://auto.mercadolibre.com.ar/MLA-1160571603-peug

/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/

...

--------------------
long name: 48
long year: 48
long km: 48
long amount: 50
long currenly: 50
long ubication: 48
long date: 48
long currenly: 48
--------------------
['https://auto.mercadolibre.com.ar/MLA-1159840697-peugeot-2022-208-16-like-pack-l20-_JM#position=1&search_layout=grid&type=item&tracking_id=e57925a6-dab3-4ba1-abfc-bbbfe279cecb', 'https://auto.mercadolibre.com.ar/MLA-1163146618-peugeot-2008-feline-no-208-cronos-308-pulse-nivus-ecosport-x-_JM#position=2&search_layout=grid&type=item&tracking_id=e57925a6-dab3-4ba1-abfc-bbbfe279cecb', 'https://auto.mercadolibre.com.ar/MLA-1163221677-peugeot-208-feline-16-tiptronic-0km-robayna-gf-_JM#position=3&search_layout=grid&type=item&tracking_id=e57925a6-dab3-4ba1-abfc-bbbfe279cecb', 'https://auto.mercadolibre.com.ar/MLA-1160761400-peugeot-207-xt-tiptronic-premium-208-307-gol-fox-ka-fit-up-x-_JM#position=4&search_layout=grid&type=item&tracking_id=e57925a6-dab3-4ba1-abfc-bbbfe279cecb', 'https://auto.mercadolibre.com.ar/MLA-1160571603-peug

2
https://autos.mercadolibre.com.ar/peugeot/peugeot-208_Desde_49_NoIndex_True


/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/

/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auto.mercadolibre.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/diego/.local/

KeyboardInterrupt: 

In [15]:
data

[('Peugeot 208 1.2 Like',
  '2022',
  19.0,
  '5.600.000',
  '$',
  'Escobar ',
  '8 días',
  'https://auto.mercadolibre.com.ar/MLA-1159840697-peugeot-2022-208-16-like-pack-l20-_JM#position=1&search_layout=grid&type=item&tracking_id=e57925a6-dab3-4ba1-abfc-bbbfe279cecb'),
 ('Peugeot 2008 1.6 Feline',
  '2017',
  62.1,
  '3.350.000',
  '$',
  'Capital Federal ',
  '1 día',
  'https://auto.mercadolibre.com.ar/MLA-1163146618-peugeot-2008-feline-no-208-cronos-308-pulse-nivus-ecosport-x-_JM#position=2&search_layout=grid&type=item&tracking_id=e57925a6-dab3-4ba1-abfc-bbbfe279cecb'),
 ('Peugeot 208 1.6l Feline Tiptronic',
  '2022',
  0.0,
  '17.000',
  'U$S',
  'Escobar ',
  '1 día',
  'https://auto.mercadolibre.com.ar/MLA-1163221677-peugeot-208-feline-16-tiptronic-0km-robayna-gf-_JM#position=3&search_layout=grid&type=item&tracking_id=e57925a6-dab3-4ba1-abfc-bbbfe279cecb'),
 ('Peugeot 207 1.6 Xt Tiptronic',
  '2012',
  89.1,
  '3.789.900',
  '$',
  'Capital Federal ',
  '6 días',
  'https://au

In [16]:
data[0]

('Peugeot 208 1.2 Like',
 '2022',
 19.0,
 '5.600.000',
 '$',
 'Escobar ',
 '8 días',
 'https://auto.mercadolibre.com.ar/MLA-1159840697-peugeot-2022-208-16-like-pack-l20-_JM#position=1&search_layout=grid&type=item&tracking_id=e57925a6-dab3-4ba1-abfc-bbbfe279cecb')

In [17]:
_save_cars(data,name_file,header)

In [18]:
import pandas as pd

In [19]:
data= pd.read_csv('Peugeot 208_cars_alfa.csv',encoding='utf-8')
data

,name,year,km,amount,currenly,ubication,date,url
0,Peugeot 208 1.2 Like,2022,19.000,5.600.000,$,Escobar,8 días,https://auto.mercadolibre.com.ar/MLA-115984069...
1,Peugeot 2008 1.6 Feline,2017,62.100,3.350.000,$,Capital Federal,1 día,https://auto.mercadolibre.com.ar/MLA-116314661...
2,Peugeot 208 1.6l Feline Tiptronic,2022,0.000,17.000,U$S,Escobar,1 día,https://auto.mercadolibre.com.ar/MLA-116322167...
3,Peugeot 207 1.6 Xt Tiptronic,2012,89.100,3.789.900,$,Capital Federal,6 días,https://auto.mercadolibre.com.ar/MLA-116076140...
4,Peugeot 208 1.6 Allure,2013,98.000,4.600.000,$,San Isidro,7 días,https://auto.mercadolibre.com.ar/MLA-116057160...
5,Peugeot 408 1.6 Sport Thp 163cv,2015,69.000,2.289.900,$,Capital Federal,8 días,https://auto.mercadolibre.com.ar/MLA-115985618...
6,Peugeot 208 1.6 Feline,2017,57.000,2.390.000,$,Capital Federal,12 días,https://auto.mercadolibre.com.ar/MLA-116159696...
7,Peugeot 208 1.6l Style,2022,0.000,3.089.900,$,Capital Federal,13 días,https://auto.mercadolibre.com.ar/MLA-115762581...
8,Peugeot 208 1.6l Feline Tiptronic,2022,0.000,3.799.000,$,La Matanza,13 días,https://auto.mercadolibre.com.ar/MLA-115736354...
9,Peugeot 208 1.6l Style Tiptronic,2022,1.000,4.395.000,$,Ezeiza,13 días,https://auto.mercadolibre.com.ar/MLA-115734298...
